In [ ]:
get predictions acc vs time of day
get predictions acc vs match mean elo
get predictions acc vs match mean number of matches
get predictions acc vs mapPlayed
get predictions acc vs entity
get predictions acc vs entityName